In [1]:
import torch
import Airfoil_DDPM_tools
import os
import pandas as pd
import numpy as np

In [2]:
data_location=os.path.join("data","ACcontext_combined_m2.csv")
data = pd.read_csv(data_location)
data.fillna(0, inplace=True)

In [3]:
print(data.shape)

(199115, 26)


In [4]:
CST = data.iloc[:, :20]
ACC = data.iloc[:, 20:]

In [5]:
print(CST.shape)

(199115, 20)


In [6]:
CST = CST.to_numpy()

In [7]:
arrays = [np.random.rand(200, 2) for _ in range(CST.shape[0])]
for i,cst in enumerate(CST):
    arrays[i] = Airfoil_DDPM_tools.CST2pointcloud(cst, 100)

point_cloud = np.stack(arrays, axis=0)  # 沿第0轴堆叠

In [8]:
print(len(arrays),len(arrays[0]),len(arrays[0][0]))

199115 200 2


In [9]:
print(point_cloud.shape)

(199115, 200, 2)


In [10]:
'''
随机打乱，这里就不采用了，到训练的时候再打乱就好
'''
ACC_Numpy = ACC.to_numpy()
shuffle_idx = torch.randperm(CST.shape[0])
shuffled_pointcloud = point_cloud[shuffle_idx]
shuffled_ACC = (ACC.to_numpy())[shuffle_idx]

In [11]:
print(shuffled_pointcloud.shape, shuffled_ACC.shape)

(199115, 200, 2) (199115, 6)


In [12]:
# 保存到文件
save_path = os.path.join("data","train_data_pointcloud.npz")
np.savez(save_path, 
         pointcloud=point_cloud, 
         ACC=ACC_Numpy)

In [13]:
loaded_data = np.load(save_path)
loaded_pointcloud = loaded_data['pointcloud']  # 形状 [B, N, 3]
loaded_ACC = loaded_data['ACC']                # 形状 [B, 6]